# Othello
A minute to learn, a lifetime to master. But if you code up a strategy, and each game takes a fraction of a second, maybe you can master it in a day or two.

This first block of code defines a bunch of structures.

In [ ]:
from typing import List, Tuple, Dict
from enum import Enum
from collections import Counter
import re
import random
from tqdm.notebook import tqdm
from IPython.core.display import display, HTML

class Place(Enum):
  EMPTY = "-"
  BLACK = "X"
  WHITE = "O"
  CHECK = "C"

class Move:
  def __init__(self, position: Tuple[int, int], pieces_flipped: List[Tuple[int,int]], turn: Place):
    # this has stuff like position, player, num pieces gained
    self.position = position
    self.gain = len(pieces_flipped)
    self.pieces_flipped = pieces_flipped
    self.turn = turn

  def __repr__(self):
    return f"Move=[({self.position[0]},{self.position[1]}),gain={self.gain}]"

  def __str__(self):
    return repr(self)

class Board:
  def __init__(self, n: int):
    self.n = n
    self._board = [[Place.EMPTY for i in range(n)] for i in range(n)]

    # starting positions
    h = int(n/2)
    self._board[h-1][h-1] = Place.WHITE
    self._board[h][h] = Place.WHITE
    self._board[h][h-1] = Place.BLACK
    self._board[h-1][h] = Place.BLACK

  def __repr__(self):
    return "\n".join(["".join([r.value for r in row]) for row in self._board])

  def html(self, move: Move):
    if not move:
      move = Move((0,0), [], None)

    s = "<table style='border-collapse: collapse;'>"
    for i,r in enumerate(self._board):
      s += "<tr>"
      for j,c in enumerate(r):
        border = "gray"
        if move.position == (i,j):
          bg_color = "#FF6347"
        else:
          bg_color = "white"
        if (i,j) in move.pieces_flipped:
          bg_color = "#90ee90"
        s += f"<td style='padding: 0; background-color: {bg_color}; width: 17px; height: 17px; border: 1px solid {border}; text-align: center; vertical-align: middle;'>{c.value if c is not Place.EMPTY else ''}</td>"
      s += "</tr>"
    s += "</table>"
    return s

  def getMoves(self, isWhiteMove: bool) -> List[Move]:
    turn = Place.WHITE if isWhiteMove else Place.BLACK

    moves = []
    for i in range(self.n):
      for j in range(self.n):
        move = self.getGain(i,j, turn)
        if move.gain > 0:
          moves.append(move)

    return moves

  def getGain(self, x: int, y: int, turn: Place) -> Move:
    """ for a given position on the board, return how many pieces 
    you could flip, or 0 if the move is not valid"""

    otherturn = Place.BLACK if turn == Place.WHITE else Place.WHITE

    if self._board[x][y] != Place.EMPTY:
      return Move((0,0), [], turn)
    
    row_list = [p.value for p in self._board[x]]
    row_list[y] = Place.CHECK.value
    col_list = [self._board[i][y].value for i in range(self.n)]
    col_list[x] = Place.CHECK.value
    
    diag_down = []
    i = x+1
    j = y+1
    diag_down.append(Place.CHECK.value)
    while i < self.n and j < self.n:
      diag_down.append(self._board[i][j].value)
      i += 1
      j += 1

    i = x-1
    j = y-1
    while i >= 0 and j >= 0:
      diag_down.insert(0, self._board[i][j].value)
      i -= 1
      j -= 1

    diag_up = []
    i = x-1
    j = y+1
    diag_up.append(Place.CHECK.value)
    while i >= 0 and j < self.n:
      diag_up.append(self._board[i][j].value)
      i -= 1
      j += 1

    i = x+1
    j = y-1
    while i < self.n and j >= 0:
      diag_up.insert(0, self._board[i][j].value)
      i += 1
      j -= 1

    row_str = "".join(row_list)
    col_str = "".join(col_list)
    diag_down_str = "".join(diag_down)
    diag_up_str = "".join(diag_up)

    debug = False
    if debug:
      print(row_str)
      print(col_str)
      print(diag_down_str)
      print(diag_up_str)

    pat_forwards = f"{Place.CHECK.value}({otherturn.value}+){turn.value}"
    pat_backwards = f"{turn.value}({otherturn.value}+){Place.CHECK.value}"
    
    poss = []

    m = re.search(pat_forwards, row_str)
    if m:
      for i in range(len(m.groups(0)[0])):
        poss.append((x, y+i+1))

    m = re.search(pat_backwards, row_str)
    if m:
      for i in range(len(m.groups(0)[0])):
        poss.append((x, y-i-1))

    m = re.search(pat_forwards, col_str)
    if m:
      for i in range(len(m.groups(0)[0])):
        poss.append((x+i+1, y))

    m = re.search(pat_backwards, col_str)
    if m:
      for i in range(len(m.groups(0)[0])):
        poss.append((x-i-1, y))
    
    m = re.search(pat_forwards, diag_down_str)
    if m:
      for i in range(len(m.groups(0)[0])):
        poss.append((x+i+1, y+i+1))

    m = re.search(pat_backwards, diag_down_str)
    if m:
      for i in range(len(m.groups(0)[0])):
        poss.append((x-i-1, y-i-1))
    
    m = re.search(pat_forwards, diag_up_str)
    if m:
      for i in range(len(m.groups(0)[0])):
        poss.append((x-i-1, y+i+1))

    m = re.search(pat_backwards, diag_up_str)
    if m:
      for i in range(len(m.groups(0)[0])):
        poss.append((x+i+1, y-i-1))

    return Move((x,y), poss, turn)

  def applyMove(self, move: Move) -> None:
    for p in move.pieces_flipped:
      self._board[p[0]][p[1]] = move.turn
    self._board[move.position[0]][move.position[1]] = move.turn

  def isComplete(self) -> bool:
    pieces_on_board = set()
    for i in range(self.n):
      for j in range(self.n):
        pieces_on_board.add(self._board[i][j])

    # possibilities: (Empty, BLACK), (EMPTY, WHITE), (BLACK, WHITE)
    return len(pieces_on_board) == 2

  def winner(self) -> Place:
    c = Counter()
    for i in range(self.n):
      for j in range(self.n):
        c[self._board[i][j]] += 1
    w = c.most_common()[0]
    if w[0] == Place.EMPTY:
      return c.most_common()[1]
    return w
    

# Strategy Functions

The following set of functions are "strategies" Each strategy takes a `move_dict`, which maps each `Move` to a priority number (larger is better), and the board object, and updates the priority values accordingly.

Update the `funcs` variable in the `whiteStrategy` and `blackStrategy` functions to set the white and black strategies respectively.

In [ ]:
def greedyMove(move_dict: Dict[Move, float], board: Board) -> None:
  for m in move_dict:
    move_dict[m] *= len(m.pieces_flipped)
  
def randomMove(move_dict: Dict[Move, float], board: Board) -> None:
  for m in move_dict:
    move_dict[m] *= random.randint(1,100)

def prioritizeCorner(move_dict: Dict[Move, float], board: Board) -> None:
  top = board.n-1
  for m in move_dict:
    if m.position in [(0,0), (top,top), (0,top), (top,0)]:
      # large number.
      move_dict[m] = 1000000

def prioritizeCenter(move_dict: Dict[Move, float], board: Board) -> None:
  # we want this to be fractional so it is symmetric
  center = (board.n / 2., board.n / 2.)
  for m in move_dict:
    move_dict[m] *= abs(m.position[0]-center[0]) + abs(m.position[1]-center[1])

def prioritizeEdges(move_dict: Dict[Move, float], board: Board) -> None:
  # we want this to be fractional so it is symmetric
  center = (board.n / 2., board.n / 2.)
  for m in move_dict:
    move_dict[m] /=  abs(m.position[0]-center[0]) + abs(m.position[1]-center[1])


def whiteStrategy(move_dict: Dict[Move, float], board: Board) -> Move:
  # Put functions in this list to determine the white strategy
  funcs = [greedyMove, prioritizeCorner]
  for f in funcs:
    f(move_dict, board)

  if debug:
    print(move_dict)
  return max(move_dict, key=move_dict.get) if len(move_dict) > 0 else None

def blackStrategy(move_dict: Dict[Move, float], board: Board) -> Move:
  # Put functions in this list to determine the black strategy
  funcs = [randomMove, prioritizeCorner]
  for f in funcs:
    f(move_dict, board)

  if debug:
    print(move_dict)
  return max(move_dict, key=move_dict.get) if len(move_dict) > 0 else None


# Main Loop

Here follows the main loop. Make sure to set `debug` and `simulations` to the way you want them.

When debugging, a board will be printed out. The O indicates a white piece, the X indicates a black piece. A tile with a red background is the piece most recently placed, and all tiles with green backgrounds are the pieces flipped during that move. Press enter to step to the next move, `q` to quit, or `c` to continue without debugging.

In [ ]:
# A board is an NxN matrix, each of which can take three values (empty, white, black).
n = 8

# Do you want to debug?
debug = False

# Change the number of simulations to get a distribution of wins/losses
simulations = 100

if debug and simulations > 1:
  print("You probably don't want more than one simulation when debugging... setting to 1.")
  simulations = 1

c = Counter()
break_all_loops = False

all_boards = []

if not debug:
  t = tqdm(total=simulations)

for i in range(simulations):
  if not debug:
    t.update()

  if break_all_loops:
    break

  if i%100 == 0:
    print(i)

  board = Board(n)
  turn = 0
  while True:
    whoseturn = Place.WHITE if turn%2==0 else Place.BLACK

    if debug:
      user_input = input(f"Turn next: {whoseturn}\n")
      if user_input in ["c", "f"]:
        debug = False
      if user_input in ["q", "quit", "end this"]:
        break_all_loops = True
        break

    move_dict = {m: 1.0 for m in board.getMoves(turn%2 == 0)}
    if whoseturn == Place.WHITE:
      move = whiteStrategy(move_dict, board)
    else:
      move = blackStrategy(move_dict, board)
    
    if move:
      board.applyMove(move)

    if debug:
      display(HTML(board.html(move)))
      print()

    if board.isComplete():
      break

    turn += 1

    if turn > 100:
      break

  w = board.winner()
  all_boards.append(board)
  c[w[0]] += 1

if not break_all_loops:
  print(c)
  print(f"White win percentage: {c[Place.WHITE] / sum(c.values()):.2%}")
  print(f"Black win percentage: {c[Place.BLACK] / sum(c.values()):.2%}")

print("Last board:")
last_board = all_boards[-1]
display(HTML(last_board.html(move)))